In [1]:
from vcdvcd import VCDVCD

vcd = VCDVCD('helloworld.vcd')

In [4]:
core_path = 'TOP.TestHarness.dut.tile.core'
voter_in = [vcd[core_path + '.multiVoter.io_in_' + str(i) + '[1997:0]'] for i in range (4)]

inject = [vcd[core_path + '.' + i + '.io_faultconf_inject'] for i in ['Rocket', 'Rocket_1', 'Rocket_2', 'Rocket_3']]

voter_sel = vcd[core_path + '.multiVoter.io_sel[1:0]']
voter_out = vcd[core_path + '.multiVoter.io_out[1997:0]']
voter_err = vcd[core_path + '.multiVoter.io_err']
fault_free_out = vcd[core_path + '.faultFreeOut[1997:0]']
clock = vcd['TOP.clock']

# number of errors is index
eval_set = [0, 0, 0, 0, 0]
eval_set_err = [0, 0, 0, 0, 0]
eval_set_out_err = [0, 0, 0, 0, 0]

start = False

for i in range(int(clock.endtime)):
    if (voter_sel[i] == "11" and clock[i] == "0"):
        faulty_core_count = sum([int(voter_in[n][i] != fault_free_out[i]) for n in range(4)])

        eval_set[faulty_core_count] += 1
        eval_set_err[faulty_core_count] += int(voter_err[i])
        eval_set_out_err[faulty_core_count] += int(voter_out[i] != fault_free_out[i])


print("total cycles:", sum(eval_set))
print("fault-free cycles:", eval_set[0], round(eval_set[0]*100/sum(eval_set), 2), '%')
print("faulty cycles:", sum(eval_set[1:]), round(sum(eval_set[1:])*100/sum(eval_set), 2), '%')
print("voter-err cycles:", sum(eval_set_err))
print("corrected cycles:", sum(eval_set[1:]) - sum(eval_set_err))
print("voter-out-err:", eval_set_out_err, sum(eval_set_out_err))

print("faulty cores:", eval_set)
print("faulty cores%:", [round(x*100/sum(eval_set), 2) for x in eval_set])
print("errors available:", eval_set_err)


total cycles: 168870
fault-free cycles: 53562 31.72 %
faulty cycles: 115308 68.28 %
voter-err cycles: 8598
corrected cycles: 106710
voter-out-err: [0, 0, 0, 5927, 680] 6607
faulty cores: [53562, 71193, 35517, 7918, 680]
faulty cores%: [31.72, 42.16, 21.03, 4.69, 0.4]
errors available: [0, 0, 0, 7918, 680]
